# Variant detector in the Ben Sira manuscripts

In [1]:
class VariantDetector:
    """
    
    """

In [8]:
import re
import os
import pprint
from bs4 import BeautifulSoup, Tag
from collatex import *

In [3]:
manuscript_files = [file for file in os.listdir("../data/") if file.startswith("ms")]

In [6]:
content = {}


for manuscript_file in manuscript_files:
    
    # Claqué au sol
    manuscript_name = manuscript_file.replace("ms_","").replace(".xml","")
    
    if manuscript_name in ["c", "2Q18"]: # Not for manuscripts c and 2q18
        continue
        
    print("====================== working on =====================")
    print(manuscript_name)
    
    # Load data
    with open(f"../data/{manuscript_file}") as f:
        manuscript = f.read()
        
    parsed = BeautifulSoup(manuscript, features="xml")
    
    if not content.get(manuscript_name):
        content[manuscript_name] = {}
    
    for link in parsed.find_all():
    
        if link.name == "chap":
            current_chapter = re.findall(r'\d+', link.text)[-1].strip()

            if not content.get(current_chapter):
                content[manuscript_name][current_chapter] = {}

        elif link.name == "text" and link.text:
            tags = [children.name for children in link]
            if "verse_nb" in tags:
                verse_nbr = link.verse_nb.text.strip()
                text_content = link.find_all(string=True, recursive=False)
                # Join text in reverse order
                text =' '.join(text_content).strip()
                content[manuscript_name][current_chapter][verse_nbr] = text
            elif verse_nbr:
                content[manuscript_name][current_chapter][verse_nbr] += ' '.join(link.find_all(text=True, recursive=False))

====================== working on =====================
masada
====================== working on =====================
b
====================== working on =====================
a
====================== working on =====================
d
====================== working on =====================
e
====================== working on =====================
f
====================== working on =====================
11QPsa


In [5]:
# Add Manuscripts C & 2Q18 to the "content" dictionary


for manuscript_file in manuscript_files:
    
    # Claqué au sol
    manuscript_name = manuscript_file.replace("ms_","").replace(".xml","")

    
    if manuscript_name in ["c", "2Q18"]: # just for manuscripts c (and 2q18)
        with open(f"../data/{manuscript_file}") as f:
            manuscript = f.read()
        parsed = BeautifulSoup(manuscript, features="xml")

        
        if not content.get(manuscript_name):
            content[manuscript_name] = {}

        for link in parsed.find_all():
          
            if link.name == "text" and link.text:
                tags = [children.name for children in link]

                if "verse_nb" in tags:
                    current = re.findall(r'\d+', link.verse_nb.text.strip())

                    if current:
                        current_chapter = current[-2]

                        if current_chapter not in content[manuscript_name]:
                            content[manuscript_name][current_chapter] = {}

                        current_verse = current[-1]

                        if current_verse not in content[manuscript_name][current_chapter]:
                            content[manuscript_name][current_chapter][current_verse] = {}
                        
                        text_content = link.find_all(string=True, recursive=False)
                        # Join text in reverse order
                        text =' '.join(text_content).strip()
                        content[manuscript_name][current_chapter][current_verse] = text
                        #print(content[manuscript_name])

In [ ]:
pprint.pprint(content)

In [15]:
#Fonction qui retourne un dictionnaire avec le manuscrit en key et une liste [chapitre, verset] en value.
# {"MSS": ["Chap, Verse", "chap1, verse"]}


def extract_keys(d, prefix=''):
    keys = []
    for key, value in d.items():
        if isinstance(value, dict):
            keys.extend(extract_keys(value, prefix=f"{prefix}{key},"))
        else:
            keys.append(f"{prefix}{key}")
    return keys



mss_chap_verses = {}
for key, value in content.items():
    keys = extract_keys(value, prefix='')
    mss_chap_verses[key] = keys

print(mss_chap_verses)

{'masada': ['39,27', '39,28ab', '39,28cd', '39,29', '39,30ab', '39,30cd', '39,31', '39,32', '39,', '40,8', '40,11', '40,12', '40,13', '40,14', '40,15', '40,16', '40,17', '40,18', '40,19ab', '40,19cd', '40,20', '40,21', '40,22', '40,23', '40,24', '40,25', '40,26ab', '40,26cd', '40,27', '40,28', '40,29ab', '40,29cd', '40,30', '41,1ab', '41,1cd', '41,2ab', '41,2cd', '41,3', '41,4ab', '41,4cd', '41,5', '41,6', '41,7', '41,8', '41,9ab', '41,9cd', '41,10', '41,11', '41,12', '41,13', '41,14bc', '41,15', '41,14a/16a', '41,16bc', '41,17', '41,18ab', '41,18c/19a', '41,19bc', '41,19d/21a', '41,21b/20a', '41,21c/20b', '41,22ab', '41,22cd', '42,1', '42,', '42,2', '42,3', '42,4ab', '42,4c/5a', '42,5bc', '42,6', '42,7', '42,8ab', '42,8cd', '42,9ab', '42,9cd', '42,10ac', '42,10bd', '42,11ab', '42,11cd', '42,11ef', '42,12', '42,13', '42,14', '42,15ab', '42,15cd', '42,16', '42,17ab', '42,17cd', '42,18ab', '42,18cd', '42,19', '42,20', '42,21ab', '42,21cd', '42,22', '42,23', '42,24', '42,25', '43,1', '43,

In [46]:
TOTAL_CHAPTERS = 51

collated_content = {}
for chapter in range(TOTAL_CHAPTERS):
    
    collated_content[str(chapter)] = {}
    
    for manuscript_id in content.keys():
        
        if content[manuscript_id].get(str(chapter)):
            
            for verse_nb, verse_content in content[manuscript_id][str(chapter)].items():
                
                if not collated_content[str(chapter)].get(verse_nb):
                    collated_content[str(chapter)][verse_nb] = {manuscript_id: verse_content}
                else:
                    collated_content[str(chapter)][verse_nb][manuscript_id] = verse_content
            

In [117]:
nbr_overlap = 0
variants = {}


for chapter in collated_content.keys():
    for verses in collated_content[chapter].keys():
        
        if len(collated_content[chapter][verses].keys()) > 1 and verses:
            
            manuscripts_to_collate = collated_content[chapter][verses]
            
            nbr_overlap += 1
            collation = Collation()

            print(f"======= chapter {chapter} ==========")
            print(f"======= verse {verses} ==========")

            for witness_name, witness_content in manuscripts_to_collate.items():
                if witness_content:
                    collation.add_plain_witness(witness_name, witness_content.replace("\u030a", "").replace("\u0307", ""))
            alignment_table = collate(collation)#, output="html2",  layout="vertical")

            if not variants.get(chapter):
                variants[chapter] = {}
            variants[chapter][verses] = {}
            for column in alignment_table.columns:
                if column.variant:
                    # Add to variants the tokens as strings       
                    for manuscript, tokens in column.tokens_per_witness.items():
                        token_strings = [token.token_string for token in tokens]
                        variants[chapter][verses].update({manuscript: " ".join(token_strings)})

======= chapter 7 ==========
======= verse 18 ==========
======= chapter 7 ==========
======= verse 19 ==========
======= chapter 7 ==========
======= verse 20 ==========
======= chapter 7 ==========
======= verse 21 ==========
======= chapter 7 ==========
======= verse 22 ==========
======= chapter 7 ==========
======= verse 23 ==========
======= chapter 7 ==========
======= verse 24 ==========
======= chapter 7 ==========
======= verse 25 ==========
======= chapter 7 ==========
======= verse 26 ==========
======= chapter 7 ==========
======= verse 29 ==========
======= chapter 7 ==========
======= verse 30 ==========
======= chapter 7 ==========
======= verse 31 ==========
======= chapter 7 ==========
======= verse 32 ==========
======= chapter 7 ==========
======= verse 33 ==========
======= chapter 7 ==========
======= verse 34 ==========
======= chapter 7 ==========
======= verse 35 ==========
======= chapter 7 ==========
======= verse 36 ==========
======= chapter 8 ==========
==

======= chapter 43 ==========
======= verse 13 ==========
======= chapter 43 ==========
======= verse 14 ==========
======= chapter 43 ==========
======= verse 15 ==========
======= chapter 43 ==========
======= verse 17cd ==========
======= chapter 43 ==========
======= verse 18 ==========
======= chapter 43 ==========
======= verse 19 ==========
======= chapter 43 ==========
======= verse 20 ==========
======= chapter 43 ==========
======= verse 21 ==========
======= chapter 43 ==========
======= verse 22 ==========
======= chapter 43 ==========
======= verse 23 ==========
======= chapter 43 ==========
======= verse 24 ==========
======= chapter 43 ==========
======= verse 25 ==========
======= chapter 43 ==========
======= verse 26 ==========
======= chapter 43 ==========
======= verse 27 ==========
======= chapter 43 ==========
======= verse 28 ==========
======= chapter 43 ==========
======= verse 29 ==========
======= chapter 43 ==========
======= verse 30 ==========
======= chap

In [221]:
# Store the variants per witness to begin variant classification
# Remove variants with fragmentation
variants_no_fragmentation = {}
for chapter, verses in variants.items():
    if not variants_no_fragmentation.get(chapter):
        variants_no_fragmentation[chapter] = {}
    for verse_nbr, variant in verses.items():
        # Merge variants
        merged_variants = " ".join(variant.values())
        if not ("[" in merged_variants or "]" in merged_variants or "׃" in merged_variants or " ̶" in merged_variants):
            if variant:
                variants_no_fragmentation[chapter][verse_nbr] = variant
            
# Re-loop in case of empty dictionary
variants_no_fragmentation = {key: value for key, value in variants_no_fragmentation.items() if value}

In [223]:
print("==============================")
print("==============================")
print("These are variants not related to fragmentation ")
print("==================================")
print("==================================")

variants_no_fragmentation

These are variants not related to fragmentation 


{'7': {'21': {'a': 'ל', 'd': 'פש אל'},
  '25': {'a': 'חברﬣ', 'd': 'חברה'},
  '31': {'a': 'צוותﬣ', 'd': 'צויתה'},
  '34': {'a': 'אבלים', 'd': 'אבילים'}},
 '8': {'5': {'a': 'ש', 'd': 'אי'},
  '6': {'a': 'נמנﬣ', 'd': 'נמנה'},
  '11': {'a': 'להושיבו כאורב לפניך', 'd': 'פניך'},
  '13': {'a': 'א \u200d ל', 'd': 'אל'},
  '15': {'a': 'א \u200d ל', 'd': 'פן'}},
 '10': {'31': {'a': 'נכבד', 'b': 'בעשרו'}},
 '11': {'3': {'a': 'דברה', 'b': 'דבורה'}, '9': {'a': 'א \u200d ל', 'b': 'אל'}},
 '15': {'1': {'a': 'י י', 'b': 'ייי'},
  '2': {'a': 'וקדמתﬣו כאם', 'b': 'וקדמתהו כ ּ אם'},
  '12': {'a': 'צורך', 'b': 'לי ח ֟ פץ'},
  '16': {'a': 'שלח', 'b': 'תשלח'},
  '17': {'a': 'ומוות אשר יחפץ ינתן', 'b': 'ומות וכל שיחפץ ית ֟ ן'},
  '19': {'a': 'איש', 'b': 'אנוש'},
  '20': {'a': 'החלים אנשי', 'b': 'למד שקרים לאנשי'}},
 '16': {'1': {'a': 'בבני', 'b': 'על בני'},
  '2': {'a': 'י י', 'b': 'ייי'},
  '4': {'a': 'וממשפחת בגדים', 'b': 'וממשפחות בוגדים'},
  '5': {'a': 'ועצ ֻ מו ֹ ת ֿ', 'b': 'ועצומות'},
  '6': {'a': 'חמﬣ'